In [1]:
import sys
import numpy as np
import random as rn
import pandas as pd
import torch
from pytorch_pretrained_bert import BertModel
from torch import nn
# from torchnlp.datasets import imdb_dataset      # --> We are using our own uploaded dataset.
from pytorch_pretrained_bert import BertTokenizer
from keras_preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
import torch.nn.functional as F
import torch.optim as optim

from IPython.display import clear_output
from transformers import AutoTokenizer, AutoModelForMaskedLM

import sqlite3
import jieba
import jieba.posseg as pseg

from sklearn.metrics import classification_report

C:\Users\HackerByeBye\Anaconda3\envs\Awesome\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rn.seed(321)
np.random.seed(321)
torch.manual_seed(321)
torch.cuda.manual_seed(321)

In [3]:
import pathlib
pathlib.Path().resolve()

WindowsPath('C:/Users/HackerByeBye/Documents/Therapy-Chatbot-Deploying-NLP/Training')

In [4]:
train_data = pd.read_csv('../train.csv')
test_data = pd.read_csv('../test.csv')

In [5]:
train_data = train_data.to_dict(orient='records')
test_data = test_data.to_dict(orient='records')
len(train_data), len(test_data)

(25476, 8491)

In [6]:
train_texts, train_labels = list(zip(*map(lambda d: (d['title'], d['label']), train_data)))
test_texts, test_labels = list(zip(*map(lambda d: (d['title'], d['label']), test_data)))
type(train_labels)
len(train_texts), len(train_labels), len(test_texts), len(test_labels)
train_labels = [x-1 for x in train_labels]
test_labels = [x-1 for x in test_labels]

In [7]:
def jiebaSlice(content,mode):
    stopword_set = []
    content = str(content)
    with open('../Analyzing/stopword.txt','r', encoding='utf-8') as stopwords:
        for stopword in stopwords:
            stopword_set.append(stopword.strip('\n'))
    
    content = content.strip('\n')
    if mode == "POSSEG":
        words = pseg.cut(content,use_paddle=True)
        slicedWords = []
        for word, flag in words:
            if word not in stopword_set:
                slicedWords.append(word)
        return slicedWords
    elif mode == "CUT_HMM":
        seg_list = jieba.cut(content,HMM=True,cut_all=True)
        slicedWords = list(seg_list)
        return slicedWords
    elif mode == "CUT_FOR_SEARCH":
        seg_list = jieba.cut_for_search(content,HMM=True)
        slicedWords = list(seg_list)
        return slicedWords
    elif mode == "NORMAL":
        seg_list = jieba.cut_for_search(content)
        slicedWords = list(seg_list)
        return slicedWords  

In [8]:
jiebaSlice(train_texts[3],'CUT_HMM')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\HACKER~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.539 seconds.
Prefix dict has been built successfully.


['遇到', '最好', '的', '卻', '不想', '還', '定下', '來']

In [9]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese', do_lower_case=True)

In [10]:
train_tokens = list(map(lambda t: ['[CLS]'] + jiebaSlice(t,'CUT_HMM') + ['[SEP]'], train_texts))

test_tokens = list(map(lambda t: ['[CLS]'] + jiebaSlice(t,'CUT_HMM') + ['[SEP]'], test_texts))

len(train_tokens), len(test_tokens)

(25476, 8491)

In [11]:
train_tokens_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, train_tokens)), maxlen=512, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, test_tokens)), maxlen=512, truncating="post", padding="post", dtype="int")

train_tokens_ids.shape, test_tokens_ids.shape

((25476, 512), (8491, 512))

In [12]:
print(train_tokens[30])

['[CLS]', '應', '該', '還', '是', '會', '有', '蠻', '多', '人', '去', 'S2o', '的', '吧', '[SEP]']


In [13]:
print(train_tokens_ids[30])

[ 101 2746 6283 6917 3221 3298 3300 6116 1914  782 1343  100 4638 1416
  102    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [14]:
train_y = np.array(train_labels)
test_y = np.array(test_labels) 
train_y.shape, test_y.shape

((25476,), (8491,))

In [15]:
train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]

In [16]:
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-chinese')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 10)
#         self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
#         proba = self.sigmoid(linear_output)
        return F.log_softmax(linear_output)

In [17]:
class BertEnhanceCNNClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertEnhanceCNNClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-chinese')
        self.dropout = nn.Dropout(dropout)
        self.cnv1 = nn.Conv1d(1,1,2,stride=2)
        self.linear = nn.Linear(384, 10)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        dropout_output = self.dropout(pooled_output)

        cnv1_output = self.cnv1(dropout_output)
        sigmoid_output = self.sigmoid(cnv1_output)

        linear_output = self.linear(sigmoid_output)

        return linear_output

In [18]:
class BertEnhanceLSTMClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertEnhanceLSTMClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-chinese')
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(768, 200, num_layers=2, bidirectional=True,dropout=0.1)
        self.softmax = nn.Softmax()
        self.fc1 = nn.Linear(400, 200)
        self.fc2 = nn.Linear(200, 10)
    
    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        dropout_output = self.dropout(pooled_output)
        h0 = torch.randn(2, 768)
        lstm_output, (hidden, cell) = self.lstm(dropout_output)
        softmax_output = self.softmax(lstm_output)
        fc1_output= self.fc1(softmax_output)
        softmax_output = self.softmax(fc1_output)
        fc2_output = self.fc2(softmax_output)
        final_output = self.softmax(fc2_output)
        return final_output

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [20]:
bert_clf = BertEnhanceCNNClassifier()
bert_clf = bert_clf.cuda()

In [21]:
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'409.718272M'

In [22]:
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'409.718272M'

In [23]:
y, x, pooled = None, None, None
torch.cuda.empty_cache()     # Clearing Cache space for fresh Model run
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'409.718272M'

In [24]:
BATCH_SIZE = 1
EPOCHS = 30

In [25]:
train_tokens_tensor = torch.tensor(train_tokens_ids)
train_y_tensor = torch.tensor(train_y.reshape(-1, 1)).float()

test_tokens_tensor = torch.tensor(test_tokens_ids)
test_y_tensor = torch.tensor(test_y.reshape(-1, 1)).float()

train_masks_tensor = torch.tensor(train_masks)
test_masks_tensor = torch.tensor(test_masks)

str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'409.718272M'

In [26]:
train_dataset = TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

test_dataset = TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

In [27]:
optimizer = Adam(bert_clf.parameters(), lr=3e-6)

In [28]:
torch.cuda.empty_cache()

In [29]:
import os 
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
epoch_loss = []
for epoch_num in range(EPOCHS):
    bert_clf.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)
        labels = torch.squeeze(labels,1)
        labels = labels.to(torch.long)
        print(str(torch.cuda.memory_allocated(device)/1000000 ) + 'M')
        logits = bert_clf(token_ids, masks)

        loss_func = nn.CrossEntropyLoss()

        batch_loss = loss_func(logits, labels) #計算loss
        train_loss += batch_loss.item()
        
        bert_clf.zero_grad() #清空前一次的gradient
        batch_loss.backward() #根據loss進行back propagation，計算gradient
        
        clip_grad_norm_(parameters=bert_clf.parameters(), max_norm=1.0)
        optimizer.step() #做gradient descent
        
        clear_output(wait=True)
        print('Epoch: ', epoch_num + 1)
        print("\r" + "{0}/{1} loss: {2} ".format(step_num, len(train_data) / BATCH_SIZE, train_loss / (step_num + 1)))
    epoch_loss.append(train_loss)

Epoch:  24
17081/25476.0 loss: 0.7040857465497629 
1700.220416M


In [ ]:
bert_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):
        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)
        labels = torch.squeeze(labels,1)
        labels = labels.to(torch.long)
        logits = bert_clf(token_ids, masks)
#         loss_func = nn.BCELoss()
        loss_func = nn.CrossEntropyLoss()
        loss = loss_func(logits, labels)
        numpy_logits = logits.cpu().detach().numpy()
        print("\r" + "{0}/{1}".format(step_num, len(test_data) / BATCH_SIZE))
        bert_predicted.append(list(numpy_logits[0, :]).index(max(list(numpy_logits[0, :]))))
        all_logits += list(numpy_logits[:, 0])

In [ ]:
print(classification_report(test_y, bert_predicted))

In [ ]:
print(epoch_loss)